# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load test data

In [2]:
dataDir = "data/2D_Truss_v1.3/conmech/"
testDir = os.path.join(dataDir, 'design_7_N_1000/')
allGraphsUnfiltered = loadConmechGraphs(testDir)

# dataDir = "data/endLoadsv1.0/conmech/"
# testDir = os.path.join(dataDir, 'design_7_N_1000/')
# allGraphsUnfiltered = loadConmechGraphs(testDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.137562
std,1.740254
min,0.004530
25%,0.010635
50%,0.016778
75%,0.032392
max,46.419552


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.019962
std,0.013149
min,0.004530
25%,0.010309
50%,0.014975
75%,0.026453
max,0.067472


## 2. Load pre-train data

In [4]:
# pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
# pretrainFiles.remove(testFile)

# allPretrainGraphs = []
# for pretrainFile in pretrainFiles:
#     designName = pretrainDir.split('/')[-1].split('_N')[0]
#     print(f'loading {designName}')
#     graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
#     graphs = filterbyDisp(graphsUnfiltered, 0.9)
#     allPretrainGraphs.extend(graphs)

# print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
# pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

In [5]:
# pretrainDirs = glob.glob('data/2D_Truss_v1.3/conmech/design_7_N_1000*/')

pretrainDirs = glob.glob(os.path.join(dataDir, '*1000/'))
pretrainDirs.remove(testDir)

allPretrainGraphs = []
for pretrainDir in pretrainDirs:
    designName = pretrainDir.split('/')[-2].split('_N')[0]
    print(f'loading from {pretrainDir}')
    graphsUnfiltered = loadConmechGraphs(pretrainDir)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading from data/2D_Truss_v1.3/conmech/design_5_N_1000/
loading from data/2D_Truss_v1.3/conmech/design_9_N_1000/
loading from data/2D_Truss_v1.3/conmech/design_6_N_1000/
loading from data/2D_Truss_v1.3/conmech/design_8_N_1000/
loaded 3600 pretraining graphs


## 3. Initial training

In [6]:
saveDir = 'results/transferLrn_endloads_des7_02/'
epochs = 100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

epoch: 0   trainLoss: 6.1295e-01   valLoss:9.3772e-01  time: 7.52e+00
epoch: 1   trainLoss: 3.1781e-01   valLoss:9.1007e-01  time: 7.26e+00
epoch: 2   trainLoss: 2.2988e-01   valLoss:8.8655e-01  time: 7.32e+00
epoch: 3   trainLoss: 1.7370e-01   valLoss:7.8663e-01  time: 7.33e+00
epoch: 4   trainLoss: 1.3490e-01   valLoss:5.0874e-01  time: 7.31e+00
epoch: 5   trainLoss: 1.3702e-01   valLoss:3.9200e-01  time: 7.32e+00
epoch: 6   trainLoss: 1.1931e-01   valLoss:9.4201e-01  time: 7.35e+00
epoch: 7   trainLoss: 1.1081e-01   valLoss:2.5473e-01  time: 7.32e+00
epoch: 8   trainLoss: 1.0649e-01   valLoss:4.6683e-01  time: 7.37e+00
epoch: 9   trainLoss: 1.0172e-01   valLoss:4.2352e-01  time: 7.39e+00
epoch: 10   trainLoss: 9.7128e-02   valLoss:9.4138e-02  time: 7.39e+00
epoch: 11   trainLoss: 7.7486e-02   valLoss:1.0351e-01  time: 7.37e+00
epoch: 12   trainLoss: 7.1469e-02   valLoss:8.4263e-02  time: 7.42e+00
epoch: 13   trainLoss: 6.3856e-02   valLoss:9.4483e-02  time: 7.38e+00
epoch: 14   trai

alt.Chart(...)

In [7]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000010,0.002060,0.088008,0.950579,NaN,NaN,NaN
test,0.000013,0.002141,0.108124,0.753640,0.926665,0.733292,0.0


## 4. Transfer learning study

In [8]:
trainDataDirs = glob.glob(os.path.join(dataDir, 'design_7*/'))
trainDataDirs.remove(testDir)

allResults = []
for trainDataDir in trainDataDirs:
    trainDataUnfiltered = loadConmechGraphs(trainDataDir)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize:05}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize:05}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                  useXFeatures=False,
                  saveDir=saveDir+f'{trainSize:05}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

loaded train set of size 906
epoch: 0   trainLoss: 8.6256e-01   valLoss:9.7420e-01  time: 8.65e+00
epoch: 1   trainLoss: 5.5629e-01   valLoss:9.5715e-01  time: 8.56e+00
epoch: 2   trainLoss: 4.3446e-01   valLoss:9.9647e-01  time: 7.51e+00
epoch: 3   trainLoss: 3.4789e-01   valLoss:1.1202e+00  time: 7.49e+00
epoch: 4   trainLoss: 3.1290e-01   valLoss:1.5436e+00  time: 7.52e+00
epoch: 5   trainLoss: 2.5940e-01   valLoss:1.6812e+00  time: 7.50e+00
epoch: 6   trainLoss: 2.3346e-01   valLoss:1.5650e+00  time: 7.54e+00
epoch: 7   trainLoss: 1.9703e-01   valLoss:1.5522e+00  time: 7.48e+00
epoch: 8   trainLoss: 1.9114e-01   valLoss:3.6080e+00  time: 7.52e+00
epoch: 9   trainLoss: 1.9495e-01   valLoss:1.4038e+00  time: 7.54e+00
epoch: 10   trainLoss: 1.7740e-01   valLoss:1.4875e+00  time: 7.47e+00
epoch: 11   trainLoss: 1.6014e-01   valLoss:1.4211e+00  time: 7.48e+00
epoch: 12   trainLoss: 1.4489e-01   valLoss:1.2753e+00  time: 7.52e+00
epoch: 13   trainLoss: 1.3787e-01   valLoss:1.6779e+00  ti

epoch: 15   trainLoss: 1.9803e-02   valLoss:5.1393e-02  time: 7.39e+00
epoch: 16   trainLoss: 2.2266e-02   valLoss:5.0471e-02  time: 7.40e+00
epoch: 17   trainLoss: 2.1903e-02   valLoss:5.7878e-02  time: 7.45e+00
epoch: 18   trainLoss: 2.0197e-02   valLoss:5.7802e-02  time: 7.39e+00
epoch: 19   trainLoss: 2.0244e-02   valLoss:2.4403e-02  time: 7.57e+00
epoch: 20   trainLoss: 1.7766e-02   valLoss:5.2994e-02  time: 7.36e+00
epoch: 21   trainLoss: 2.0940e-02   valLoss:2.4069e-02  time: 7.34e+00
epoch: 22   trainLoss: 1.7526e-02   valLoss:4.9558e-02  time: 7.40e+00
epoch: 23   trainLoss: 1.7983e-02   valLoss:2.1987e-02  time: 7.41e+00
epoch: 24   trainLoss: 1.3650e-02   valLoss:6.6442e-02  time: 7.40e+00
epoch: 25   trainLoss: 1.5327e-02   valLoss:2.0136e-02  time: 7.48e+00
epoch: 26   trainLoss: 1.6475e-02   valLoss:2.9214e-02  time: 7.52e+00
epoch: 27   trainLoss: 2.2225e-02   valLoss:2.9037e-02  time: 7.57e+00
epoch: 28   trainLoss: 1.4874e-02   valLoss:3.3974e-02  time: 7.84e+00
epoch:

FileNotFoundError: [Errno 2] No such file or directory: 'results/transferLrn_endloads_des7_02/testResults.csv'

In [ ]:
# df = pd.DataFrame(allResults)
df = pd.read_csv('results/transferLrn_endloads_des7_02/testResults.csv')
df

In [ ]:
df = df.replace('Fresh', 'GCN')
df = df.replace('Transfer learning', 'GCN with transfer learning')

In [ ]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE', axis=alt.Axis(format='.1e')),
    color='Model',
    tooltip=['Model', 'mse']
).properties(width=400, height=200, title='Transfer learning - group 7')